# Import

In [27]:
import numpy as np
import pandas as pd
import datetime

import sys, os
sys.path.insert(0, os.path.abspath('scripts'))
import rus_table_parser

# Data parsing

In [28]:
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 9
# MONTH = 6

# Save of "https://coronavirus-monitor.ru/coronavirus-v-rossii/" page
path = f'html_data/{DAY}-{MONTH}/Коронавирус в России. Онлайн карта распространения коронавируса в России..html'
parse_df = rus_table_parser.run_parsing(path)
parse_df

,Confirmed,Date,Deaths,Recovered,Region/City
0,201221.0,2020-06-11,3138.0,115769.0,Москва
1,47181.0,2020-06-11,644.0,14187.0,Московская область
2,19769.0,2020-06-11,520.0,8864.0,Санкт-Петербург
3,13341.0,2020-06-11,152.0,6219.0,Нижегородская область
4,8506.0,2020-06-11,48.0,3790.0,Свердловская область
...,...,...,...,...,...
80,264.0,2020-06-11,0.0,71.0,Сахалинская область
81,212.0,2020-06-11,0.0,111.0,Республика Алтай
82,178.0,2020-06-11,3.0,151.0,Севастополь
83,114.0,2020-06-11,1.0,70.0,Чукотский автономный округ


# Preparing parsed data

In [29]:
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
5755,2020-06-10,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,197.0,2.0,264.0,3821.0,23.0,1468.0
5756,2020-06-10,Чеченская Республика,Chechen Republic,95.0,16.0,0.0,49.0,1400.0,16.0,1073.0
5757,2020-06-10,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,2.0,0.0,1.0,112.0,1.0,69.0
5758,2020-06-10,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,54.0,0.0,7.0,2887.0,13.0,1482.0
5759,2020-06-10,Ярославская область,Yaroslavl region,76.0,63.0,0.0,142.0,3844.0,15.0,1452.0


In [30]:
# Create day-columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

In [31]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b')
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

In [32]:
# Rename regions
rename_dict = {
    'Татарстан' : 'Республика Татарстан',
    'Башкортостан' : 'Республика Башкортостан',
    'Чувашская Республика': 'Республика Чувашия',
    'Камчатский край' : 'Камчатский край',
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
    'Республика Алтай' : 'Республика Алтай'
}

parse_df['Region/City'] = parse_df['Region/City'].replace(rename_dict)

In [33]:
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# Minus a day from now if needed
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [34]:
# Fit day-columns
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Confirmed'].max()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Deaths'].max()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Recovered'].max()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [35]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [36]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 502436.0 Day-Confirmed: 8665.0
Russia Deaths: 6559.0 Day-Deaths: 174.0
Russia Recovered: 261173.0 Day-Recovered: 8350.0


# Data saving

In [37]:
# Save full snapshot of current day
parse_df.to_csv(f'covid19-russia-cases-upd-full.csv', index=False)

In [38]:
# Filter regions without changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [39]:
# Save parsed data for future use
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd.csv', index=False)